In [1]:
import cryo
import polars as pl
import binascii
import web3
import json
from eth_abi import decode
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd

In [2]:
# Functions
def bytes_to_hexstr(b: any) -> str:
    if isinstance(b,list):
        return [bytes_to_hexstr(a) for a in b]
    return '0x' + b.hex()

In [3]:
# Function Signatures 4 bytes
getBlocknumber_4b = '42cbb15c'

In [4]:
txs = cryo.collect('transactions', blocks=['latest'], rpc ='https://eth.merkle.io')

In [5]:
txs.head()

block_number,transaction_index,transaction_hash,nonce,from_address,to_address,value_binary,value_string,value_f64,input,gas_limit,gas_used,gas_price,transaction_type,max_priority_fee_per_gas,max_fee_per_gas,success,n_input_bytes,n_input_zero_bytes,n_input_nonzero_bytes,chain_id
u32,u64,binary,u64,binary,binary,binary,str,f64,binary,u64,u64,u64,u32,u64,u64,bool,u32,u32,u32,u64
22089468,0,"b""\xae\x1a:\x05\xb1\xae\x05\xd9\x0aW\xf0\x1f\xa8\x20+\xa6\xbf\xd1LX\x08_\x01.\x0b\x0aWY2W*\x9d""",7799,"b""\xbdx5\x13U-\xc7\xb6\x88\x14\xbb{}\x10\\x19\xa0\xf4\xd3A""","b""\xef\xf6\xcb\x8baI\x99\xd10\xe57u\x1e\xe9\x97$\xd0\x1a\xa1g""","b""\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x002\x84""","""12932""",12932.0,"b""\xa9\x0b\xc6\xeb\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00g\xdcKc\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00M\xd2d\x82s\x8b\xe6\xc0l1Fz""…",151692,93117,22109456838,2,20932553059,22262321339,true,260,174,86,1
22089468,1,"b""Z\x15\x0e\xa7\xa84v\x93\x86T\xef\xe77\xda\xde\xc1g\x09Y]\x1eL\xc6\x91\x10Op\x94\xe3\x96\xcfQ""",6777,"b"")-\xa0\xbf\xf7/\x03\x83\xa78&\x91\x92\xfd\x01R\x9b\xa0T3""","b""\xef\xf6\xcb\x8baI\x99\xd10\xe57u\x1e\xe9\x97$\xd0\x1a\xa1g""","b""\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x001L""","""12620""",12620.0,"b""\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00g\xdcKc\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00IXd\xca\xd6\xcf`\xe9~\xfc\xd3\xf1""…",168119,104068,3423630352,2,2246726573,3576494853,true,260,171,89,1
22089468,2,"b""\x8e\xc6\xb6e\x85]\x954\xbcM\x1b=\xc5\x95\xf2~]\xd4\x96sUHS\x8aO7S~\xcdT\x8d\x01""",8618,"b""\x9d~ad{`\xb4\xc238\xed\x95\x97\x88\xc7,\xd7b\x1f\xd4""","b""\xef\xf6\xcb\x8baI\x99\xd10\xe57u\x1e\xe9\x97$\xd0\x1a\xa1g""","b""\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0026""","""12854""",12854.0,"b""\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00g\xdcKc\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00}\xf7\xc8O/\x9d\xce\xf3\xc0\x81>S""…",217818,137201,2085129170,2,908225391,2237993671,true,260,175,85,1
22089468,3,"b""\x0b\xa0Y\xc3}\xea\x91_\xa1I\x83\x15W\x05\xf7j\x8d\x09k\x03\x8d\x8e\x1a$\x9c\xe1\xd3\x93\xe4\xd6\xd1x""",1507,"b""r~\xe9\xa0M&\x05hI\xe2\x98\x10Tt\x9biw\x8c_1""","b""S\xb5\x9f\xb7\x90cg\xc8\xd4KiJ/\x82\xfe\xddI\x11E\x8c""","b""\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x07Z\xa7n\xaf`\x00""","""2070000000000000""",2.0700e15,"b""B\xb5O\xee\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00""…",410192,281396,1676903779,2,500000000,2329140042,true,1508,1334,174,1
22089468,4,"b""\x14r\xf5\x8d\xfb\x89\xb6pL\xb2\xeb\x82i\xaf\xd26\xba\xbfWH<V\xc9A\xcf@=\xeb7\xba\x0d\xa9""",15,"b""\x9f\x89\xa68\xd9\x9cSJ\xd8D'""\xaa\x90\xed\x93/)\x8c\xe9""","b""f\xa9\x89<\xc0}\x91\xd9VD\xae\xdd\x05\xd0?\x95\xe1\xdb\xa8\xaf""","b""\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00#\x86\xf2o\xc1\x00\x00""","""10000000000000000""",1.0000e16,"b""5\x93VL\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00`\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00""…",232914,160074,3176903779,2,2000000000,3526894338,true,965,797,168,1


## Balance

In [202]:
# ERC20s
USDT_ADDR ="0xdac17f958d2ee523a2206206994597c13d831ec7"
USDC_ADDR="0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48" 
# WALLET = Binance 18
RANDOM_WALLET="0xf70da97812CB96acDF810712Aa562db8dfA3dbEF" 
# block 
BLOCKS="21922375"


In [203]:
usdc_balance = cryo.collect('erc20_balances', blocks=[BLOCKS], contract=['0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'], address=['0xf70da97812CB96acDF810712Aa562db8dfA3dbEF'], rpc='https://eth.merkle.io')

In [204]:
usdc_balance

block_number,erc20,address,balance_binary,balance_string,balance_f64,chain_id
u32,binary,binary,binary,str,f64,u64
21922375,"b""\xa0\xb8i\x91\xc6!\x8b6\xc1\xd1\x9dJ.\x9e\xb0\xce6\x06\xebH""","b""\xf7\x0d\xa9x\x12\xcb\x96\xac\xdf\x81\x07\x12\xaaV-\xb8\xdf\xa3\xdb\xef""","b""\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00n\x1a)\x8f\x80""","""472885333888""",4.7289e11,1


In [205]:
output_addr=usdc_balance['erc20'][0]

In [206]:
output_addr.hex()

'a0b86991c6218b36c1d19d4a2e9eb0ce3606eb48'

In [207]:
usdc_balance['address'][0].hex()

'f70da97812cb96acdf810712aa562db8dfa3dbef'

In [208]:
usdc_balance['balance_binary'][0]

b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00n\x1a)\x8f\x80'

In [209]:
balance= usdc_balance['balance_binary'][0]

In [210]:
int((balance).hex(),16)

472885333888

In [211]:
usdc_balance['balance_string'][0]

'472885333888'

In [212]:
balance_raw = decode(['uint256'], usdc_balance['balance_binary'][0])

In [213]:
balance_raw

(472885333888,)

In [214]:
 balance_raw[0] / 10**6

472885.333888

In [215]:
# BLOCKS="21922375" >> usdc 472,885.333888

## Aave Pools Data

In [216]:
#  AAVE
# https://etherscan.io/address/0x87870bca3f3fd6335c3f4ce8392d69350b4fa4e2#readProxyContract

In [6]:
MULTICALL3_ADDRESS = '0xcA11bde05977b3631167028862bE2a173976CA11'
MULTICALL3_ABI=json.loads('[{"inputs":[{"internalType":"bool","name":"requireSuccess","type":"bool"},{"components":[{"internalType":"address","name":"target","type":"address"},{"internalType":"bytes","name":"callData","type":"bytes"}],"internalType":"struct Multicall3.Call[]","name":"calls","type":"tuple[]"}],"name":"tryAggregate","outputs":[{"components":[{"internalType":"bool","name":"success","type":"bool"},{"internalType":"bytes","name":"returnData","type":"bytes"}],"internalType":"struct Multicall3.Result[]","name":"returnData","type":"tuple[]"}],"stateMutability":"payable","type":"function"}]')

In [7]:
AAVE_POOL_PROVIDER_ADDRESS = "0x2f39d218133AFaB8F2B819B1066c7E434Ad94E9e"
AAVE_POOL_DATA_PROVIDER_ADDRESS = "0x7B4EB56E7CD4b454BA8ff71E4518426369a138a3"
RATE_STRATEGY_V2= "0x9ec6F08190DeA04A54f8Afc53Db96134e5E3FdFB"
DAI_ADDRESS = "0x6b175474e89094c44da98b954eedeac495271d0f"  # DAI on Ethereum Mainnet
WETH_ADDRESS = '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'  # WETH
USDC_ADDRESS = '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'   # USDC

In [8]:
AAVE_POOL_DATA_PROVIDER_ABI = json.loads('[{"inputs":[{"internalType":"address","name":"asset","type":"address"}],"name":"getReserveData","outputs":[{"internalType":"uint256","name":"unbacked","type":"uint256"},{"internalType":"uint256","name":"accruedToTreasuryScaled","type":"uint256"},{"internalType":"uint256","name":"totalAToken","type":"uint256"},{"internalType":"uint256","name":"totalStableDebt","type":"uint256"},{"internalType":"uint256","name":"totalVariableDebt","type":"uint256"},{"internalType":"uint256","name":"liquidityRate","type":"uint256"},{"internalType":"uint256","name":"variableBorrowRate","type":"uint256"},{"internalType":"uint256","name":"stableBorrowRate","type":"uint256"},{"internalType":"uint256","name":"averageStableBorrowRate","type":"uint256"},{"internalType":"uint256","name":"liquidityIndex","type":"uint256"},{"internalType":"uint256","name":"variableBorrowIndex","type":"uint256"},{"internalType":"uint40","name":"lastUpdateTimestamp","type":"uint40"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"asset","type":"address"}],"name":"getReserveTokensAddresses","outputs":[{"internalType":"address","name":"aTokenAddress","type":"address"},{"internalType":"address","name":"stableDebtTokenAddress","type":"address"},{"internalType":"address","name":"variableDebtTokenAddress","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getAllReservesTokens","outputs":[{"components":[{"internalType":"string","name":"symbol","type":"string"},{"internalType":"address","name":"tokenAddress","type":"address"}],"internalType":"struct IPoolDataProvider.TokenData[]","name":"","type":"tuple[]"}],"stateMutability":"view","type":"function"}]')


In [9]:
# Funct Sig
getAllReservesTokens_4b='b316ff89'
getReserveData_4b='35ea6a75'
getReserveTokensAddresses_4b='d2493b6c'

In [221]:
# getReserveData(address) method Response ]
#   unbacked   uint256 :  0
#   accruedToTreasuryScaled   uint256 :  23618787780361774928
#   totalAToken   uint256 :  2039801843604753838311618
#   totalStableDebt   uint256 :  0
#   totalVariableDebt   uint256 :  1764755511852628454833269
#   liquidityRate   uint256 :  19086348066645041248157072
#   variableBorrowRate   uint256 :  25954495065727705440586566
#   stableBorrowRate   uint256 :  0
#   averageStableBorrowRate   uint256 :  0
#   liquidityIndex   uint256 :  1040724217363078276332975414
#   variableBorrowIndex   uint256 :  1064692592486700566822429320
#   lastUpdateTimestamp   uint40 :  1740762059

# unbacked, accruedToTreasuryScaled, totalAToken, totalStableDebt, totalVariableDebt, liquidityRate,variableBorrowRate,stableBorrowRate,averageStableBorrowRate,liquidityIndex,variableBorrowIndex,lastUpdateTimestamp

# output_types = ['int256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint40']

In [10]:
# web3 instance, function from web3py
w3 = web3.Web3()
a3 = w3.eth.contract(address = AAVE_POOL_DATA_PROVIDER_ADDRESS, abi=AAVE_POOL_DATA_PROVIDER_ABI)
m3 = w3.eth.contract(address = MULTICALL3_ADDRESS, abi=MULTICALL3_ABI)

In [11]:
tokens = [
    '0x6B175474E89094C44Da98b954EedeAC495271d0F',  # DAI
    '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2',  # WETH
    '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'   # USDC
]

In [12]:
calls = []
for token in tokens:
    # Encode the call to getReserveData for this token
    calldata = a3.encode_abi("getReserveData", args=[token])
    calls.append([AAVE_POOL_DATA_PROVIDER_ADDRESS, calldata])

In [13]:
calls

[['0x7B4EB56E7CD4b454BA8ff71E4518426369a138a3',
  '0x35ea6a750000000000000000000000006b175474e89094c44da98b954eedeac495271d0f'],
 ['0x7B4EB56E7CD4b454BA8ff71E4518426369a138a3',
  '0x35ea6a75000000000000000000000000c02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'],
 ['0x7B4EB56E7CD4b454BA8ff71E4518426369a138a3',
  '0x35ea6a75000000000000000000000000a0b86991c6218b36c1d19d4a2e9eb0ce3606eb48']]

In [14]:
# result = m3.functions.tryAggregate(False, calls).call()
calldata_aave_pool = m3.encode_abi("tryAggregate", args=[False, calls])

In [15]:
cryo_kwargs = {
    'rpc': 'https://eth.merkle.io',
    'blocks': ['latest'], 
}
            
eth_call_df = cryo.collect(
    'eth_calls',
    to_address = [MULTICALL3_ADDRESS],
    call_data=[calldata_aave_pool],
     output_format="polars",
    **cryo_kwargs,
)

In [16]:
calldata = eth_call_df['output_data'][0]

In [17]:
decoded_data = decode(['(bool,bytes)[]'] , calldata)[0]

In [18]:
decoded_data

((True,
  b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00Ro\xd1\x8e~\xf5-\xa4\x8b\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x8b\x96`\xe4\xd5\x02\xa9o\xc4|\x0c\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00h\xe8.\xec\rF\xb11\xa0\xf3\xf0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x18\xc1\xbc\xaa&\xdb\x837\r\xe7t\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00+\xeb\xf6\x8c\x86\x91\xff\xbb\x82c"\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x0

In [19]:
token_data = {
    'DAI_RESERVE_DATA': [],
    'WETH_RESERVE_DATA': [],
    'USDC_RESERVE_DATA': []
}

#### DAI Pools Data

In [20]:
dai_reserve_data = decoded_data[0]

In [21]:
data_dai=decode(['int256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint40'],dai_reserve_data[1])

In [22]:
token_data['DAI_RESERVE_DATA'] = list(data_dai)

In [23]:
token_data

{'DAI_RESERVE_DATA': [0,
  1520690391928456193163,
  168750831275383515617983500,
  0,
  126824739823503677002413040,
  29929116650559191288506228,
  53098114391886893424993058,
  0,
  0,
  1120821053109081998735848233,
  1177334864585937558368019691,
  1742490431],
 'WETH_RESERVE_DATA': [],
 'USDC_RESERVE_DATA': []}

In [236]:
# [unbacked, accruedToTreasuryScaled, totalAToken, totalStableDebt, totalVariableDebt, liquidityRate,variableBorrowRate,stableBorrowRate,averageStableBorrowRate,liquidityIndex,variableBorrowIndex,lastUpdateTimestamp
# ]=decode(['int256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint40'],dai_reserve_data[1])

####  WETH Pools Data

In [24]:
weth_reserve_data = decoded_data[1]

In [25]:
data_weth=decode(['int256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint40'],weth_reserve_data[1])

In [26]:
token_data['WETH_RESERVE_DATA'] = list(data_weth)

In [240]:
# [unbacked, accruedToTreasuryScaled, totalAToken, totalStableDebt, totalVariableDebt, liquidityRate,variableBorrowRate,stableBorrowRate,averageStableBorrowRate,liquidityIndex,variableBorrowIndex,lastUpdateTimestamp
# ]=decode(['int256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint40'],weth_reserve_data[1])

In [28]:
token_data

{'DAI_RESERVE_DATA': [0,
  1520690391928456193163,
  168750831275383515617983500,
  0,
  126824739823503677002413040,
  29929116650559191288506228,
  53098114391886893424993058,
  0,
  0,
  1120821053109081998735848233,
  1177334864585937558368019691,
  1742490431],
 'WETH_RESERVE_DATA': [0,
  6671845738186803039,
  2176763763204976640972125,
  0,
  1897250395172238692816251,
  19371537969474904127465005,
  26147683269255568188308537,
  0,
  0,
  1041957754427318071639746589,
  1066356404212933774092800783,
  1742490455],
 'USDC_RESERVE_DATA': []}

####  USDC Pools Data

In [29]:
usdc_reserve_data = decoded_data[2]

In [30]:
data_usdc=decode(['int256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint40'],usdc_reserve_data[1])

In [31]:
token_data['USDC_RESERVE_DATA'] = list(data_usdc)

In [245]:
# [unbacked, accruedToTreasuryScaled, totalAToken, totalStableDebt, totalVariableDebt, liquidityRate,variableBorrowRate,stableBorrowRate,averageStableBorrowRate,liquidityIndex,variableBorrowIndex,lastUpdateTimestamp
# ]=decode(['int256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint256', 'uint40'],usdc_reserve_data[1])

In [32]:
token_data

{'DAI_RESERVE_DATA': [0,
  1520690391928456193163,
  168750831275383515617983500,
  0,
  126824739823503677002413040,
  29929116650559191288506228,
  53098114391886893424993058,
  0,
  0,
  1120821053109081998735848233,
  1177334864585937558368019691,
  1742490431],
 'WETH_RESERVE_DATA': [0,
  6671845738186803039,
  2176763763204976640972125,
  0,
  1897250395172238692816251,
  19371537969474904127465005,
  26147683269255568188308537,
  0,
  0,
  1041957754427318071639746589,
  1066356404212933774092800783,
  1742490455],
 'USDC_RESERVE_DATA': [0,
  6317850508,
  2829283686153193,
  0,
  1755295559939976,
  24474470427386899984418637,
  43832694084167005788937934,
  0,
  0,
  1123978061549572496945632615,
  1163129777333551630278719149,
  1742490383]}

### Aave Pool Data

In [33]:
field_names = [
    'unbacked', 'accruedToTreasuryScaled', 'totalAToken', 'totalStableDebt', 
    'totalVariableDebt', 'liquidityRate', 'variableBorrowRate', 'stableBorrowRate', 
    'averageStableBorrowRate', 'liquidityIndex', 'variableBorrowIndex', 'lastUpdateTimestamp'
]

In [34]:
df = pd.DataFrame(token_data, index=field_names)

In [35]:
# Convert token amounts - DAI and WETH (18 decimals)
df.loc['totalAToken', 'DAI_RESERVE_DATA'] = df.loc['totalAToken', 'DAI_RESERVE_DATA'] / 1e18
df.loc['totalAToken', 'WETH_RESERVE_DATA'] = df.loc['totalAToken', 'WETH_RESERVE_DATA'] / 1e18
df.loc['totalVariableDebt', 'DAI_RESERVE_DATA'] = df.loc['totalVariableDebt', 'DAI_RESERVE_DATA'] / 1e18
df.loc['totalVariableDebt', 'WETH_RESERVE_DATA'] = df.loc['totalVariableDebt', 'WETH_RESERVE_DATA'] / 1e18
# Convert token amounts - USDC (6 decimals)
df.loc['totalAToken', 'USDC_RESERVE_DATA'] = df.loc['totalAToken', 'USDC_RESERVE_DATA'] / 1e6
df.loc['totalVariableDebt', 'USDC_RESERVE_DATA'] = df.loc['totalVariableDebt', 'USDC_RESERVE_DATA'] / 1e6

In [36]:
# Convert interest rates from ray (10^27) to percentages
df.loc['liquidityRate'] = (df.loc['liquidityRate'] / 1e27)
df.loc['variableBorrowRate'] = (df.loc['variableBorrowRate'] / 1e27)

In [37]:
# Format timestamps (this does convert to strings, but that's probably what you want for timestamps)
from datetime import datetime
df.loc['lastUpdateTimestamp'] = df.loc['lastUpdateTimestamp'].apply(
    lambda x: datetime.fromtimestamp(int(x)).strftime('%Y-%m-%d %H:%M:%S')
)

In [38]:
df.loc['variableBorrowRate']

DAI_RESERVE_DATA     0.053098
WETH_RESERVE_DATA    0.026148
USDC_RESERVE_DATA    0.043833
Name: variableBorrowRate, dtype: object

In [131]:
# pd.set_option('display.float_format', '{:,.4f}'.format)

In [39]:
df

,DAI_RESERVE_DATA,WETH_RESERVE_DATA,USDC_RESERVE_DATA
unbacked,0,0,0
accruedToTreasuryScaled,1520690391928456193163,6671845738186803039,6317850508
totalAToken,168750831.275384,2176763.763205,2829283686.153193
totalStableDebt,0,0,0
totalVariableDebt,126824739.823504,1897250.395172,1755295559.939976
liquidityRate,0.029929,0.019372,0.024474
variableBorrowRate,0.053098,0.026148,0.043833
stableBorrowRate,0,0,0
averageStableBorrowRate,0,0,0
liquidityIndex,1120821053109081998735848233,1041957754427318071639746589,1123978061549572496945632615


In [44]:
def display_formatted_df(df):
    # Make a copy to avoid modifying the original
    display_df = df.copy()
    
    # Format token amounts with commas and 2 decimal places
    display_df.loc['totalAToken'] = display_df.loc['totalAToken'].apply(lambda x: f"{x:,.2f}")
    display_df.loc['totalVariableDebt'] = display_df.loc['totalVariableDebt'].apply(lambda x: f"{x:,.2f}")
    
    return display_df

In [46]:
display_formatted_df(df)

,DAI_RESERVE_DATA,WETH_RESERVE_DATA,USDC_RESERVE_DATA
unbacked,0,0,0
accruedToTreasuryScaled,1520690391928456193163,6671845738186803039,6317850508
totalAToken,"168,750,831.28","2,176,763.76","2,829,283,686.15"
totalStableDebt,0,0,0
totalVariableDebt,"126,824,739.82","1,897,250.40","1,755,295,559.94"
liquidityRate,0.029929,0.019372,0.024474
variableBorrowRate,0.053098,0.026148,0.043833
stableBorrowRate,0,0,0
averageStableBorrowRate,0,0,0
liquidityIndex,1120821053109081998735848233,1041957754427318071639746589,1123978061549572496945632615


In [ ]:
# Utilization Rate = = TotalBorrowed / TotalSupply
TotBorrow = 

In [49]:
df.loc['totalVariableDebt', 'DAI_RESERVE_DATA']/df.loc['totalAToken', 'DAI_RESERVE_DATA']

0.7515503115746974

In [51]:
# getInterestRateData(address).varaibleRateSlope1
# RATE_STRATEGY_V2.getInterestRateData(DAI_ADDRESS)

In [61]:
DAI_ADDRESS = w3.to_checksum_address("0x6B175474E89094C44Da98b954EedeAC495271d0F")

In [63]:
print(w3.is_connected())  # Should return Tru

False


In [52]:
RATE_STRATEGY_V2_ABI = json.loads('[{"inputs":[{"internalType":"address","name":"reserve","type":"address"}],"name":"getInterestRateData","outputs":[{"components":[{"internalType":"uint256","name":"optimalUsageRatio","type":"uint256"},{"internalType":"uint256","name":"baseVariableBorrowRate","type":"uint256"},{"internalType":"uint256","name":"variableRateSlope1","type":"uint256"},{"internalType":"uint256","name":"variableRateSlope2","type":"uint256"}],"internalType":"struct IDefaultInterestRateStrategyV2.InterestRateDataRay","name":"","type":"tuple"}],"stateMutability":"view","type":"function"} ]')

In [62]:
# Load the contract
contract = w3.eth.contract(address=RATE_STRATEGY_V2, abi=RATE_STRATEGY_V2_ABI)

# Call the function
interest_rate_data = contract.functions.getInterestRateData(DAI_ADDRESS).call()

CannotHandleRequest: Could not discover provider while making request: method:eth_chainId
params:()


In [42]:
# Convert token amounts - DAI and WETH (18 decimals)
# df.loc['totalAToken', 'DAI_RESERVE_DATA'] = df.loc['totalAToken', 'DAI_RESERVE_DATA'] / 1e18

In [43]:
# Apply the unit conversions with different decimals per token
# For DAI and WETH (18 decimals)
readable_df.loc['totalAToken', 'DAI_RESERVE_DATA'] = readable_df.loc['totalAToken', 'DAI_RESERVE_DATA'] / 1e18
readable_df.loc['totalAToken', 'WETH_RESERVE_DATA'] = readable_df.loc['totalAToken', 'WETH_RESERVE_DATA'] / 1e18
readable_df.loc['totalVariableDebt', 'DAI_RESERVE_DATA'] = readable_df.loc['totalVariableDebt', 'DAI_RESERVE_DATA'] / 1e18
readable_df.loc['totalVariableDebt', 'WETH_RESERVE_DATA'] = readable_df.loc['totalVariableDebt', 'WETH_RESERVE_DATA'] / 1e18

# For USDC (6 decimals)
readable_df.loc['totalAToken', 'USDC_RESERVE_DATA'] = readable_df.loc['totalAToken', 'USDC_RESERVE_DATA'] / 1e6
readable_df.loc['totalVariableDebt', 'USDC_RESERVE_DATA'] = readable_df.loc['totalVariableDebt', 'USDC_RESERVE_DATA'] / 1e6

# Convert rates from ray (10^27) to percentage for all tokens (as numeric values)
readable_df.loc['liquidityRate'] = readable_df.loc['liquidityRate'] / 1e27 * 100
readable_df.loc['variableBorrowRate'] = readable_df.loc['variableBorrowRate'] / 1e27 * 100

# Convert timestamp to datetime but keep the original values for calculations if needed
from datetime import datetime
timestamp_values = readable_df.loc['lastUpdateTimestamp'].copy()
readable_df.loc['lastUpdateTimestamp'] = readable_df.loc['lastUpdateTimestamp'].apply(
    lambda x: datetime.fromtimestamp(int(x)).strftime('%Y-%m-%d %H:%M:%S')
)


ValueError: invalid literal for int() with base 10: '2025-03-20 17:07:11'

In [53]:
readable_df.loc['lastUpdateTimestamp'] = readable_df.loc['lastUpdateTimestamp'].apply(
    lambda x: datetime.fromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S')
)

In [54]:
readable_df

,DAI_RESERVE_DATA,WETH_RESERVE_DATA,USDC_RESERVE_DATA
unbacked,0,0,0
accruedToTreasuryScaled,1272631322734850943859,5527230188022745360,5239782724
totalAToken,168749992958756623159816609,2176891299909195728593931,2829884560059423
totalStableDebt,0,0,0
totalVariableDebt,126825046777808676903510963,1898468140944825244961765,1752279879182154
liquidityRate,29929652764255528955771044,19394162087811683453066052,24380113945504062751438188
variableBorrowRate,53098589957204005475672965,26162947820225160659630962,43748118346190638871715653
stableBorrowRate,0,0,0
averageStableBorrowRate,0,0,0
liquidityIndex,1120815513262587206006570762,1041954506011921037712383387,1123973729543179762214097964


In [55]:
readable_df.loc['totalAToken', 'DAI_RESERVE_DATA'] = readable_df.loc['totalAToken', 'DAI_RESERVE_DATA'] / 1e18
readable_df.loc['totalAToken', 'WETH_RESERVE_DATA'] = readable_df.loc['totalAToken', 'WETH_RESERVE_DATA'] / 1e18
readable_df.loc['totalVariableDebt', 'DAI_RESERVE_DATA'] = readable_df.loc['totalVariableDebt', 'DAI_RESERVE_DATA'] / 1e18
readable_df.loc['totalVariableDebt', 'WETH_RESERVE_DATA'] = readable_df.loc['totalVariableDebt', 'WETH_RESERVE_DATA'] / 1e18

In [56]:
readable_df.loc['totalAToken', 'USDC_RESERVE_DATA'] = readable_df.loc['totalAToken', 'USDC_RESERVE_DATA'] / 1e6
readable_df.loc['totalVariableDebt', 'USDC_RESERVE_DATA'] = readable_df.loc['totalVariableDebt', 'USDC_RESERVE_DATA'] / 1e6

In [57]:
readable_df.loc['totalAToken'] = readable_df.loc['totalAToken'].apply(lambda x: f"{float(x):,.4f}" if isinstance(x, (int, float, str)) else x)
readable_df.loc['totalVariableDebt'] = readable_df.loc['totalVariableDebt'].apply(lambda x: f"{float(x):,.4f}" if isinstance(x, (int, float, str)) else x)


In [58]:
# Convert rates from ray (10^27) to percentage for all tokens
readable_df.loc['liquidityRate'] = readable_df.loc['liquidityRate'] / 1e27 * 100
readable_df.loc['variableBorrowRate'] = readable_df.loc['variableBorrowRate'] / 1e27 * 100

In [59]:
# Format the rate fields with 4 decimal places and add % sign
readable_df.loc['liquidityRate'] = readable_df.loc['liquidityRate'].apply(lambda x: f"{float(x):.4f}%" if isinstance(x, (int, float, str)) else x)
readable_df.loc['variableBorrowRate'] = readable_df.loc['variableBorrowRate'].apply(lambda x: f"{float(x):.4f}%" if isinstance(x, (int, float, str)) else x)

In [402]:
# readable_df.loc['totalAToken'] = df.loc['totalAToken'].apply(lambda x: f"{x:,.4f}")
# readable_df.loc['totalVariableDebt'] = df.loc['totalVariableDebt'].apply(lambda x: f"{x:,.4f}")
# readable_df.loc['liquidityRate'] = df.loc['liquidityRate'].apply(lambda x: f"{x:.4f}%")
# readable_df.loc['variableBorrowRate'] = df.loc['variableBorrowRate'].apply(lambda x: f"{x:.4f}%")

In [60]:
readable_df

,DAI_RESERVE_DATA,WETH_RESERVE_DATA,USDC_RESERVE_DATA
unbacked,0,0,0
accruedToTreasuryScaled,1272631322734850943859,5527230188022745360,5239782724
totalAToken,"168,749,992.9588","2,176,891.2999","2,829,884,560.0594"
totalStableDebt,0,0,0
totalVariableDebt,"126,825,046.7778","1,898,468.1409","1,752,279,879.1822"
liquidityRate,2.9930%,1.9394%,2.4380%
variableBorrowRate,5.3099%,2.6163%,4.3748%
stableBorrowRate,0,0,0
averageStableBorrowRate,0,0,0
liquidityIndex,1120815513262587206006570762,1041954506011921037712383387,1123973729543179762214097964


In [61]:
df

,DAI_RESERVE_DATA,WETH_RESERVE_DATA,USDC_RESERVE_DATA
unbacked,0,0,0
accruedToTreasuryScaled,1272631322734850943859,5527230188022745360,5239782724
totalAToken,"168,749,992.9588","2,176,891.2999","2,829,884,560.0594"
totalStableDebt,0,0,0
totalVariableDebt,"126,825,046.7778","1,898,468.1409","1,752,279,879.1822"
liquidityRate,2.9930%,1.9394%,2.4380%
variableBorrowRate,5.3099%,2.6163%,4.3748%
stableBorrowRate,0,0,0
averageStableBorrowRate,0,0,0
liquidityIndex,1120815513262587206006570762,1041954506011921037712383387,1123973729543179762214097964


In [62]:
readable_df = df.rename(columns={
    'DAI_RESERVE_DATA': 'DAI_POOL',
    'WETH_RESERVE_DATA': 'WETH_POOL',
    'USDC_RESERVE_DATA': 'USDC_POOL'
})

In [63]:
readable_df

,DAI_POOL,WETH_POOL,USDC_POOL
unbacked,0,0,0
accruedToTreasuryScaled,1272631322734850943859,5527230188022745360,5239782724
totalAToken,"168,749,992.9588","2,176,891.2999","2,829,884,560.0594"
totalStableDebt,0,0,0
totalVariableDebt,"126,825,046.7778","1,898,468.1409","1,752,279,879.1822"
liquidityRate,2.9930%,1.9394%,2.4380%
variableBorrowRate,5.3099%,2.6163%,4.3748%
stableBorrowRate,0,0,0
averageStableBorrowRate,0,0,0
liquidityIndex,1120815513262587206006570762,1041954506011921037712383387,1123973729543179762214097964


In [65]:
readable_df.loc['totalVariableDebt', 'USDC_POOL']

'1,752,279,879.1822'

In [74]:
aave_pools_df = readable_df.copy()

In [75]:
aave_pools_df.loc['liquidityRate'] = df.loc['liquidityRate'].apply(
    lambda x: float(x.replace('%', '')) if isinstance(x, str) else x
)
aave_pools_df.loc['variableBorrowRate'] = df.loc['variableBorrowRate'].apply(
    lambda x: float(x.replace('%', '')) if isinstance(x, str) else x
)

In [72]:
aave_pools_df.loc['variableBorrowRate'] = df.loc['variableBorrowRate'].apply(
    lambda x: float(x.replace('%', '')) if isinstance(x, str) else x
)

In [76]:
aave_pools_df

,DAI_POOL,WETH_POOL,USDC_POOL
unbacked,0,0,0
accruedToTreasuryScaled,1272631322734850943859,5527230188022745360,5239782724
totalAToken,"168,749,992.9588","2,176,891.2999","2,829,884,560.0594"
totalStableDebt,0,0,0
totalVariableDebt,"126,825,046.7778","1,898,468.1409","1,752,279,879.1822"
liquidityRate,NaN,NaN,NaN
variableBorrowRate,NaN,NaN,NaN
stableBorrowRate,0,0,0
averageStableBorrowRate,0,0,0
liquidityIndex,1120815513262587206006570762,1041954506011921037712383387,1123973729543179762214097964
